## Практическое задание к уроку № 8 по теме "Снижение размерности данных".

#### Задание 1.  
*Можно ли отобрать наиболее значимые признаки с помощью PCA? Ответ объясните.*

Думаю, что нельзя. Метод PCA никак не учитывает целевую переменную и влияние признаков на неё.  
Он стремится сократить размерность, не оказывая сильного влияния на дисперсию исходных данных.  
Он как бы сжимает данные в данные меньшей размерности и не отбирает признаки, а преобразовывает их.

#### Задание 2.  
*Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd().  
Применить к данным на уроке и сравнить ответы.*

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

Возьмём данные из урока:

In [2]:
iris = load_iris()
X = iris.data
X.shape

(150, 4)

Стандартизируем признаки:

In [3]:
class StandardScaler:
    
    def __init__(self):
        self.mean = None
        self.std = None
    
    def fit(self, X):
        self.mean = X.mean(axis=0)
        self.std = X.std(axis=0)
    
    def transform(self, X):
        return (X - self.mean) / self.std
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

Матрица весов, полученная на уроке, которую нужно получить нам:

[[ 0.52106591 -0.37741762]  
[-0.26934744 -0.92329566]  
[ 0.5804131  -0.02449161]  
[ 0.56485654 -0.06694199]]

Разложим матрицу:

In [5]:
U, s, W = np.linalg.svd(X)

Матрица V:

In [6]:
V = W.T
V

array([[ 0.52106591, -0.37741762,  0.71956635,  0.26128628],
       [-0.26934744, -0.92329566, -0.24438178, -0.12350962],
       [ 0.5804131 , -0.02449161, -0.14212637, -0.80144925],
       [ 0.56485654, -0.06694199, -0.63427274,  0.52359713]])

Сингулярные числа матрицы X:

In [7]:
s

array([20.92306556, 11.7091661 ,  4.69185798,  1.76273239])

Сформируем новую матрицу весов W из столбцов матрицы V, соответствующих двум наибольшим сингулярным числам:

In [8]:
W = V[:, :2]
W

array([[ 0.52106591, -0.37741762],
       [-0.26934744, -0.92329566],
       [ 0.5804131 , -0.02449161],
       [ 0.56485654, -0.06694199]])

Эта матрица весов равна матрице с урока, где мы пользовались методом PCA.

Итоговая матрица будет равна:

In [9]:
Z = X @ W
Z.shape

(150, 2)

#### Задание 3.  
*Обучить любую модель классификации (из рассмотренных в курсе) на датасете IRIS до применения PCA и после него.  
Сравнить качество классификации по отложенной выборке.*

Сделаем разбивку данных на выборки:

In [10]:
X, y = load_iris(return_X_y=True)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [12]:
X_train.shape, X_test.shape

((120, 4), (30, 4))

Вместо PCA воспользуемся SVD-разложением, которое только что делали:

In [13]:
def process(X):
    U, s, W = np.linalg.svd(X)
    W = W.T[:, :2]
    return X @ W

В качестве модели классификации воспользуемся kNN с предыдущего урока:

In [14]:
def l2_norm(x1, x2):
    return np.sqrt(sum((x1 - x2)**2))

In [15]:
def knn(x_train, y_train, x_test, k, weighted=None):
    
    answers = []
    for x in x_test:
        test_distances = []
            
        for i in range(len(x_train)):
            
            # расчет расстояния от классифицируемого объекта до
            # объекта обучающей выборки
            distance = l2_norm(x, x_train[i])
            
            # Записываем в список значение расстояния и ответа на объекте обучающей выборки
            test_distances.append((distance, y_train[i]))
        
        # отберем k ближайших соседей
        k_neighbours = sorted(test_distances)[:k]
                
        # создаем словарь со всеми возможными классами
        classes = {class_item: 0 for class_item in np.unique(y_train)}
        
        # Подсчитаем суммарный вес разных классов среди ближайших соседей
        for idx, d in enumerate(k_neighbours):
            if weighted == 'order':
                weight = 1 / (idx + 1)
            elif weighted == 'distance':
                weight = 0.5**d[0]
            else:
                weight = 1
            classes[d[1]] += weight
            
        # Записываем в список ответов наиболее весомый класс
        answers.append(max(classes, key=classes.get))
    return answers

Для оценки качества классификации будем использовать метрику accuracy:

In [16]:
def calc_accuracy(y, y_pred):
    y = np.array(y)
    y_pred = np.array(y_pred)
    
    return (y == y_pred).sum() / y.shape[0]

В этой задаче будем использовать нормализацию вместо стандартизации, так как при стандартизации мы получаем некорректные признаки  
после снижения размерности. Скорее всего, это связано с тем, что стандартизация не сохраняет расстояния, и когда  
мы пытаемся снизить размерность тестовой выборки, которая была стандартизирована по обучающей выборке, получаем некорректную матрицу данных.

In [17]:
class NormalScaler:
    
    def __init__(self):
        self.min = None
        self.max = None
    
    def fit(self, X):
        self.min = X.min(axis=0)
        self.max = X.max(axis=0)
    
    def transform(self, X):
        return (X - self.min) / (self.max - self.min)
    
    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

Нормализуем выборки:

In [18]:
scaler = NormalScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Сделаем выборки со сниженной размерностью:

In [19]:
X_train_processed = process(X_train)
X_test_processed = process(X_test)

In [20]:
X_train_processed.shape, X_test_processed.shape

((120, 2), (30, 2))

Сравним модели:

In [21]:
y_pred = knn(X_train, y_train, X_test, k=3)
y_pred_processed = knn(X_train_processed, y_train, X_test_processed, k=3)

print(f'Модель kNN, обученная на всех 4 признаках:\n'
      f'Accuracy - {calc_accuracy(y_test, y_pred):.2f}\n')
print(f'Модель kNN, обученная на 2 признаках после снижения размерности:\n'
      f'Accuracy - {calc_accuracy(y_test, y_pred_processed):.2f}')

Модель kNN, обученная на всех 4 признаках:
Accuracy - 1.00

Модель kNN, обученная на 2 признаках после снижения размерности:
Accuracy - 0.90


После снижения размерности с 4-х до 2-х признаков модель kNN снизила точность на тестовой выборке на 0,1.  
Посмотрим как отреагирует дерево решений из урока № 4 на снижение размерности:

In [22]:
import import_ipynb
from Lesson_4_mytree import DecisionTree

importing Jupyter notebook from Lesson_4_mytree.ipynb


In [23]:
model = DecisionTree()
model_processed = DecisionTree()

model.fit(X_train, y_train)
model_processed.fit(X_train_processed, y_train)

In [24]:
y_pred = model.predict(X_test)
y_pred_processed = model_processed.predict(X_test_processed)

print(f'Модель дерева решений, обученная на всех 4 признаках:\n'
      f'Accuracy - {calc_accuracy(y_test, y_pred):.2f}\n')
print(f'Модель дерева решений, обученная на 2 признаках после снижения размерности:\n'
      f'Accuracy - {calc_accuracy(y_test, y_pred_processed):.2f}')

Модель дерева решений, обученная на всех 4 признаках:
Accuracy - 0.97

Модель дерева решений, обученная на 2 признаках после снижения размерности:
Accuracy - 0.87


Дерево решений также снизило точность на 0,1.  
Снижение размерности данных привело к потере части информации, и такие простые модели как те, которые мы протестировали, показали существенное снижение точности.  
Выбор более сложных моделей, либо подстройка гиперпараметров наших моделей может позволить сократить отставание.